In [5]:
#最终版
from scipy.spatial import distance
from imutils import face_utils
import numpy as np
import dlib
import cv2
import tkinter as tk
from tkinter import filedialog

def eyeAspectRatio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear


file_path = ''
while file_path == '':
    file_path = filedialog.askopenfilenames(title='请选择一个视频', initialdir='resources/eye/')
choose_file = file_path[0]

cap = cv2.VideoCapture(choose_file)  # 视频路径

# 使用Dlib的面部检测器和形状预测器
detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# 眼睛位置
(l_eye_start, l_eye_end) = face_utils.FACIAL_LANDMARKS_68_IDXS["left_eye"]
(r_eye_start, r_eye_end) = face_utils.FACIAL_LANDMARKS_68_IDXS["right_eye"]

# 判断闭眼的阈值和帧数
thresh = 0.25
frame_check = 5
blink_counter = 0
alarm_on = False

while True:
    ret, frame = cap.read()
    if not ret:
        break
    # 转换当前帧为灰度图像
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 自适应直方图均衡化
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe_gray = clahe.apply(gray)

    # 对CLAHE结果应用高斯模糊，以减少噪声
    blur_sigma = 2  # 高斯核的标准偏差，在不同图像上可能需要调整
    gaussian_blur = cv2.GaussianBlur(clahe_gray, (5, 5), blur_sigma)

    # 使用Canny算法进行边缘检测
    low_threshold = 50  # Canny边缘检测的低阈值
    high_threshold = 150  # Canny边缘检测的高阈值
    canny_edges = cv2.Canny(gaussian_blur, low_threshold, high_threshold)

    # 进行形态学变换 - 闭操作
    kernel = np.ones((5, 5), np.uint8)
    closing = cv2.morphologyEx(canny_edges, cv2.MORPH_CLOSE, kernel)

    # 将闭操作后的边缘与原CLAHE图像进行叠加
    # 转换闭操作后的边缘图像的数据类型，使其可以与原图像进行加权叠加
    closing_for_combination = cv2.convertScaleAbs(closing)
    enhanced_edges = cv2.addWeighted(clahe_gray, 0.7, closing_for_combination, 0.3, 0)
    
    faces = detect(gray, 0)
    for face in faces:
        shape = predict(gray, face)
        shape = face_utils.shape_to_np(shape)

        leftEye = shape[l_eye_start:l_eye_end]
        rightEye = shape[r_eye_start:r_eye_end]
        leftEAR = eyeAspectRatio(leftEye)
        rightEAR = eyeAspectRatio(rightEye)

        ear = (leftEAR + rightEAR) / 2.0

        if ear < thresh:
            blink_counter += 1
            if blink_counter >= frame_check:
                if not alarm_on:
                    alarm_on = True
                cv2.putText(frame, "DROWSINESS ALERT!", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            blink_counter = 0
            alarm_on = False

        cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

In [9]:
from scipy.spatial import distance
from imutils import face_utils
import imutils
import dlib
import numpy as np
import cv2
import tkinter as tk
from tkinter import filedialog

def eyeAspectRatio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear


# 判断闭眼的阈值和帧数
thresh = 0.32
frame_check = 20
blink_counter = 0
alarm_on = False


detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")  # Dat file is the crux of the code

# 眼睛对应的点
(l_eye_start, l_eye_end) = face_utils.FACIAL_LANDMARKS_68_IDXS["left_eye"]
(r_eye_start, r_eye_end) = face_utils.FACIAL_LANDMARKS_68_IDXS["right_eye"]

# 眉毛对应的点
(l_eyeb_start, l_eyeb_end) = face_utils.FACIAL_LANDMARKS_68_IDXS["left_eyebrow"]
(r_eyeb_start, r_eyeb_end) = face_utils.FACIAL_LANDMARKS_68_IDXS["right_eyebrow"]

# 视频选择对话框
file_path = ''
while file_path == '':
    file_path = filedialog.askopenfilenames(title='请选择一个视频', initialdir='resources/eye/')
choose_file = file_path[0]

cap = cv2.VideoCapture(choose_file)  # 视频路径

# 准备输出视频流
fps = cap.get(cv2.CAP_PROP_FPS)
size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter('../results/eye/eye_closed_res_' + choose_file[-6:], fourcc, fps, size)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 图像预处理
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 自适应直方图均衡化
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe_gray = clahe.apply(gray)

    # 对CLAHE结果应用高斯模糊，以减少噪声
    blur_sigma = 2  # 高斯核的标准偏差，在不同图像上可能需要调整
    gaussian_blur = cv2.GaussianBlur(clahe_gray, (5, 5), blur_sigma)

    # 使用Canny算法进行边缘检测
    low_threshold = 50  # Canny边缘检测的低阈值
    high_threshold = 150  # Canny边缘检测的高阈值
    canny_edges = cv2.Canny(gaussian_blur, low_threshold, high_threshold)

    # 进行形态学变换 - 闭操作
    kernel = np.ones((5, 5), np.uint8)
    closing = cv2.morphologyEx(canny_edges, cv2.MORPH_CLOSE, kernel)

    # 人脸检测
    faces = detect(gray, 0)
    for face in faces:
        # 人脸关键点预测
        face_68_point = predict(gray, face)
        face_68_point = face_utils.shape_to_np(face_68_point)  # 转化为numpy数组

        # 关键点划分
        left_eye = face_68_point[l_eye_start:l_eye_end]  # 左眼
        right_eye = face_68_point[r_eye_start:r_eye_end]  # 右眼

        # 左右眼的ear系数计算
        left_EAR = eyeAspectRatio(left_eye)
        right_EAR = eyeAspectRatio(right_eye)
        ear = (left_EAR + right_EAR) / 2.0

        # 找人脸轮廓
        left = face.left()
        top = face.top()
        right = face.right()
        bottom = face.bottom()

        up_offset = int((bottom - top) * 0.2)
        down_offset = int((bottom - top) * 0.4)

        cv2.rectangle(frame, (left, top), (right, bottom - down_offset), (0, 255, 0), 3)

        # 判断睁闭眼
        if ear < thresh:
            blink_counter += 1
            if blink_counter >= frame_check:
                if not alarm_on:
                    alarm_on = True
                cv2.putText(frame, "DROWSINESS ALERT!", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            # if yawn_alerted:
            #     yawn_alerted = False
            blink_counter = 0
            #blink_counter = 0
            alarm_on = False

    out.write(frame)
    cv2.imshow("Frame", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord(" "):
        cv2.waitKey(0)
    elif key == ord("q"):
        break

cap.release()
out.release()
cv2.destroyAllWindows()